## Load libraries

In [1]:
import pandas as pd
import numpy as np
import helper as h

import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error

import eli5

## Upload data

In [2]:
df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(h.parse_price)

df_test = pd.read_hdf("../input/test_data.h5")

## Combine data
Combine `df_train` and` df_test` and save the result to `df`.

In [3]:
df = pd.concat([df_train, df_test])
print(df.shape, df.shape)

(34180, 8) (34180, 8)


In [4]:
df.sample(5)

,geo_block,breadcrumbs,price,owner,params,date,user_block,id
5551,"[г. Москва, Чертаново Центральное, ул Академик...","[Москва, Чертаново Центральное, м. Улица Акаде...",NaN,[],"{'Охрана:': 'консьерж', 'Тип здания:': 'Моноли...","[6 января 2018, 278, (+1 за сегодня), Обновлен...","[<div class=""block-user__name"">Щебланин Владим...",59227
6614,"[г. Москва, г. Москва]","[Москва, МЦК ЗИЛ]",5.820000,[],"{'Охрана:': 'предусмотрена', 'Тип здания:': 'М...","[10 августа 2018, 44, (+2 за сегодня), Обновле...","[<div class=""block-user__name"">БЕСТ Новострой<...",66737
11665,"[Новая Москва, п. Внуковское, г. Москва, Новая...","[Москва, Новая Москва, п. Внуковское, МЦК Куту...",8.830000,[Переделкино Ближнее],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[17 мая, 3, (+1 за сегодня)]",[],11503
8019,"[г. Москва, г. Москва]","[Москва, МЦК Кутузовская]",8.079523,[],"{'Лифт:': 'да', 'Охрана:': 'консьерж', 'Тип зд...","[18 февраля, 14, (+1 за сегодня), Обновлено 14...","[<div class=""block-user__name""></div>, <div cl...",6763
8620,"[г. Москва, Ховрино, г. Москва, Ховрино]","[Москва, Ховрино, м. Ховрино, МЦК Коптево]",NaN,[],"{'Лифт:': 'да', 'Тип здания:': 'Монолитное', '...","[10 мая, 2, (+1 за сегодня), Обновлено 16 мая]","[<div class=""block-user__name"">MR-Group</div>,...",15106


### Categorization of the Params feature

In [5]:
params = df["params"].apply(pd.Series)
params = params.fillna(-1)

if "Охрана:" not in df:  #prevents params from being added again 
    df = pd.concat([df, params], axis=1)
    
    obj_feats = params.select_dtypes(object).columns

    for feat in obj_feats:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]

cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

['Охрана:_cat',
 'Тип здания:_cat',
 'Тип объекта:_cat',
 'Количество корпусов:_cat',
 'Тип объявления:_cat',
 'Застройщик:_cat',
 'Общая площадь:_cat',
 'Дата публикации:_cat',
 'Количество комнат:_cat',
 'Парковка:_cat',
 'Дата  обновления:_cat',
 'Количество этажей:_cat',
 'Сдача:_cat',
 'Комиссия агенту:_cat',
 'Высота потолков:_cat',
 'Этаж:_cat',
 'Этап строительства:_cat',
 'Новостройка:_cat',
 'Количество квартир:_cat',
 'Класс жилья:_cat',
 'Адрес:_cat',
 'Лифт:_cat',
 'Вид из окна:_cat',
 'Возможна ипотека:_cat',
 'Год постройки:_cat',
 'Этажность:_cat',
 'Мусоропровод:_cat',
 'Ремонт:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Тип санузла:_cat',
 'Тип балкона:_cat',
 'Мебель на кухне:_cat',
 'Холодильник:_cat',
 'Интернет:_cat',
 'Мебель:_cat',
 'Телефон:_cat',
 'Свободная планировка:_cat',
 'Управляющая компания:_cat',
 'Количество подъездов:_cat',
 'Тип дома:_cat',
 'Тип комнат:_cat',
 'Покрытие пола:_cat',
 'Серия:_cat',
 'Тип перекрытия:_cat',
 'Возможен торг:

### Features of area 

In [6]:
def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

df["area"] = df["Общая площадь:"].map(h.parse_area)
df["kitchen_area"] = df["Площадь кухни:"].map(parse_area)

### model evaluation 

Two functions: `check_model`, `check_log_model`. 

In [7]:
def check_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()
    df_test = df[ df["price"].isnull() ].copy()

    X_train = df_train[feats]
    y_train = df_train["price"]
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)
    return np.mean(scores), np.std(scores)


def check_log_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        model = xgb.XGBRegressor(max_depth=5, n_estimators=50, random_state=0)
        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)

    return np.mean(scores), np.std(scores)

## Features of area 
Selected features: `geo_block` i `breadcrumbs`.

### geo_block

In [8]:
df["geo_block"].values

array([list(['г. Москва', 'Лианозово', 'г. Москва', 'Лианозово']),
       list(['г. Москва', 'Ховрино', 'г. Москва', 'Ховрино']),
       list(['г. Москва', 'ул Лобачевского', 'г. Москва', 'ул Лобачевского']),
       ..., list(['г. Москва', 'Ховрино', 'г. Москва', 'Ховрино']),
       list(['Новая Москва', 'п. Внуковское', 'г. Москва', 'Новая Москва', 'п. Внуковское', 'г. Москва']),
       list(['г. Москва', 'Ярославский', 'ул Красная Сосна', 'г. Москва', 'Ярославский', 'ул Красная Сосна'])],
      dtype=object)

Removal of duplicates, because in the lines we have two elements repeated twice. 

In [9]:
df["geo_block_norm"] = df["geo_block"].map(lambda x: x[:int(len(x)/2)])

df["geo_block_norm"]

0                            [г. Москва, Лианозово]
1                              [г. Москва, Ховрино]
2                      [г. Москва, ул Лобачевского]
3                  [г. Москва, Ховрино, ул Дыбенко]
4                          [г. Москва, Даниловский]
                            ...                    
11443                                   [г. Москва]
11444                [г. Москва, Хорошёво-Мнёвники]
11445                          [г. Москва, Ховрино]
11446      [Новая Москва, п. Внуковское, г. Москва]
11447    [г. Москва, Ярославский, ул Красная Сосна]
Name: geo_block_norm, Length: 34180, dtype: object

Create `DataFrame`.

In [10]:
geo_block = (
df["geo_block"]
    .map(lambda x: x[:int(len(x)/2) ])
    .map(lambda x: {"geo_block_{}".format(idx):val for idx,val in enumerate(x) })
    .apply(pd.Series)
)

geo_block

,geo_block_0,geo_block_1,geo_block_2,geo_block_3,geo_block_4,geo_block_5
0,г. Москва,Лианозово,NaN,NaN,NaN,NaN
1,г. Москва,Ховрино,NaN,NaN,NaN,NaN
2,г. Москва,ул Лобачевского,NaN,NaN,NaN,NaN
3,г. Москва,Ховрино,ул Дыбенко,NaN,NaN,NaN
4,г. Москва,Даниловский,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
11443,г. Москва,NaN,NaN,NaN,NaN,NaN
11444,г. Москва,Хорошёво-Мнёвники,NaN,NaN,NaN,NaN
11445,г. Москва,Ховрино,NaN,NaN,NaN,NaN
11446,Новая Москва,п. Внуковское,г. Москва,NaN,NaN,NaN


Categorization of the `geo_block` feature

In [11]:

for feat in geo_block.columns:
    print(feat)
    print(geo_block[feat].value_counts().head(20))
    print("="*100)
    print()
    

    df["{}_cat".format(feat)] = geo_block[feat].factorize()[0]

geo_block_0
г. Москва             29049
Новая Москва           4531
г. Зеленоград           318
п. Свиблово             138
п. Некрасовка           134
п. Новобратцевский        3
пгт Акулово               1
п. Крюково                1
п. Главмосстроя           1
п. Алабушево              1
д. Сколково               1
п. Подушкино              1
п. Рублево                1
Name: geo_block_0, dtype: int64

geo_block_1
п. Сосенское              2522
Хорошёво-Мнёвники         1142
Филёвский Парк             945
п. Московский              919
Ховрино                    829
Раменки                    811
Люблино                    666
Некрасовка                 654
Кунцево                    537
Орехово-Борисово Южное     534
ул Дыбенко                 525
Выхино-Жулебино            496
Зябликово                  492
ул Лобачевского            442
п. Внуковское              430
Рязанский                  425
Марьино                    407
Пресненский                378
Лефортово            

In [12]:
geo_cat_feats = [x for x in df.columns if "geo_block" in x and "_cat" in x]
geo_cat_feats

['geo_block_0_cat',
 'geo_block_1_cat',
 'geo_block_2_cat',
 'geo_block_3_cat',
 'geo_block_4_cat',
 'geo_block_5_cat']

### Check result

In [13]:
feats = ["area", "kitchen_area"] + geo_cat_feats + cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.25358809911422, 0.3150982251982633)

### Breadcrumbs 


In [14]:
breadcrumbs = (
    df["breadcrumbs"]
        .map(lambda x: {"breadcrumbs_{}".format(idx):val for idx,val in enumerate(x) })
        .apply(pd.Series)
)

breadcrumbs

,breadcrumbs_0,breadcrumbs_1,breadcrumbs_2,breadcrumbs_3,breadcrumbs_4,breadcrumbs_5,breadcrumbs_6,breadcrumbs_7
0,Москва,Лианозово,м. Алтуфьево,МЦК Лихоборы,NaN,NaN,NaN,NaN
1,Москва,Ховрино,м. Ховрино,МЦК Коптево,NaN,NaN,NaN,NaN
2,Москва,МЦК Кутузовская,ул. Лобачевского,NaN,NaN,NaN,NaN,NaN
3,Москва,Ховрино,м. Ховрино,МЦК Коптево,ул. Дыбенко,NaN,NaN,NaN
4,Москва,Даниловский,м. Автозаводская,МЦК Автозаводская,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11443,Москва,МЦК Кутузовская,NaN,NaN,NaN,NaN,NaN,NaN
11444,Москва,Хорошёво-Мнёвники,м. Октябрьское поле,МЦК Зорге,NaN,NaN,NaN,NaN
11445,Москва,Ховрино,м. Ховрино,МЦК Коптево,NaN,NaN,NaN,NaN
11446,Москва,Новая Москва,п. Внуковское,МЦК Кутузовская,NaN,NaN,NaN,NaN


Categorization of the `breadcrumbs` feature

In [15]:
for feat in breadcrumbs.columns:
    print(feat)
    print(breadcrumbs[feat].value_counts().head(20))
    print("="*100)
    print()
    

    df["{}_cat".format(feat)] = breadcrumbs[feat].factorize()[0]

breadcrumbs_0
Москва    34180
Name: breadcrumbs_0, dtype: int64

breadcrumbs_1
Новая Москва              4531
МЦК Лужники               1245
МЦК Кутузовская           1237
Хорошёво-Мнёвники         1142
Филёвский Парк             945
МЦК Крымская               932
МЦК Шелепиха               842
Ховрино                    829
Раменки                    811
МЦК Коптево                800
Люблино                    666
МЦК Хорошёво               633
Некрасовка                 608
МЦК Нижегородская          564
МЦК Лихоборы               549
Кунцево                    537
Орехово-Борисово Южное     534
Выхино-Жулебино            494
Зябликово                  492
МЦК Ростокино              453
Name: breadcrumbs_1, dtype: int64

breadcrumbs_2
п. Сосенское             2522
п. Московский             919
м. Октябрьское поле       864
м. Выхино                 856
м. Ховрино                829
м. Братиславская          649
м. Рязанский проспект     600
м. Пятницкое шоссе        558
м. Фили     

In [16]:
breadcrumbs_cat_feats = [x for x in df.columns if "breadcrumbs" in x and "_cat" in x]
breadcrumbs_cat_feats

['breadcrumbs_0_cat',
 'breadcrumbs_1_cat',
 'breadcrumbs_2_cat',
 'breadcrumbs_3_cat',
 'breadcrumbs_4_cat',
 'breadcrumbs_5_cat',
 'breadcrumbs_6_cat',
 'breadcrumbs_7_cat']

A selection of the 7 best features from `breadcrumbs`. 

In [17]:
beast_feats = ['Общая площадь:_cat',
 'Класс жилья:_cat',
 'Этаж:_cat',
 'Количество комнат:_cat',
 'Высота потолков:_cat',
 'Дата публикации:_cat',
 'Тип балкона:_cat',
 'Застройщик:_cat',
 'Парковка:_cat',
 'Количество квартир:_cat',
 'Количество этажей:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Охрана:_cat',
 'Тип здания:_cat',
 'Год постройки:_cat',
 'Сдача:_cat'
              ]

### Check result

In [18]:
feats = ["area", "kitchen_area"] + geo_cat_feats + beast_feats + breadcrumbs_cat_feats;
check_log_model(df, feats, xgb.XGBRegressor(max_depth=10, n_estimators=50, learning_rate=0.3, random_state=0))

(3.085983458562901, 0.33152654352557553)

## Summary of reasult

In [19]:
feats = ["area", "kitchen_area"] + geo_cat_feats + beast_feats + breadcrumbs_cat_feats

df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

X_train = df_train[feats]
y_train = df_train["price"]
y_log_train = np.log(y_train)

X_test = df_test[feats]

model = xgb.XGBRegressor(max_depth=10, n_estimators=50, learning_rate=0.3, random_state=0)
model.fit(X_train, y_log_train)
y_log_pred = model.predict(X_test)
y_pred = np.exp(y_log_pred)


## Creat a result file 

In [21]:
#save the data to a file 

df_test["price"] = y_pred
df_test[ ["id", "price"] ].to_csv("../output/xgb_location_log_area.csv", index=False)